In [1]:
from datasets import load_dataset, Dataset
import json
from pathlib import Path
import os

/home/idea_user/anaconda3/envs/icl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
dataset_name = "ProofWriter"
split = "test"
data_path = "../data"
datafile_path = os.path.join(data_path, dataset_name, f'{split}.json')
print(datafile_path)

ds_dict = load_dataset("json", data_files={split:datafile_path})
print(ds_dict)
raw_dataset = ds_dict[split]
print(raw_dataset)

../data/ProofWriter/test.json


In [32]:
from pandas import DataFrame
start = 0
end =8
chunk = raw_dataset.select(range(start, end))
print(chunk[0])
# print(raw_dataset[0])
# print(chunk["question"])
print(type(chunk))
print(type(raw_dataset))
# for example in raw_dataset:
#     print(example)
#     break
# for example in chunk:
#     print(example)

{'id': 'ProofWriter_RelNeg-OWA-D5-136_Q1', 'context': 'The bear does not chase the lion. The bear visits the lion. The bear visits the tiger. The lion chases the bear. The lion chases the tiger. The lion visits the bear. The rabbit chases the tiger. The rabbit is big. The rabbit is not green. The rabbit is young. The rabbit visits the tiger. The tiger chases the rabbit. The tiger is rough. The tiger visits the bear. If something chases the bear and it sees the rabbit then the bear does not visit the rabbit. If something chases the lion then the lion is rough. If the rabbit chases the tiger and the tiger visits the rabbit then the rabbit is not big. If something chases the lion then the lion does not visit the tiger. If something chases the rabbit and it chases the tiger then the rabbit chases the bear. If something is young and it does not see the rabbit then it sees the bear. If something visits the lion then it is young. If the tiger visits the bear and the bear is big then the tiger

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
import torch
from vllm import LLM, SamplingParams

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
model_path = "../llms/Qwen2.5-7B-Instruct"
# tokenizer = AutoTokenizer.from_pretrained(model_path, padding_side="left")
# model = AutoModelForCausalLM.from_pretrained(model_path,device_map="auto")


/home/idea_user/anaconda3/envs/icl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 12-28 21:10:19 [__init__.py:216] Automatically detected platform cuda.


In [2]:
tokenizer = AutoTokenizer.from_pretrained(model_path, padding_side="left")

hf_model = AutoModelForCausalLM.from_pretrained(model_path,device_map="auto")

Loading checkpoint shards: 100%|██████████| 4/4 [00:09<00:00,  2.34s/it]


In [3]:
vllm_model = LLM(model=model_path, tokenizer=model_path,tensor_parallel_size= torch.cuda.device_count(),max_model_len=4096,dtype="float16",trust_remote_code=True,gpu_memory_utilization=0.5)


INFO 12-28 21:10:30 [utils.py:328] non-default args: {'tokenizer': '../llms/Qwen2.5-7B-Instruct', 'trust_remote_code': True, 'dtype': 'float16', 'max_model_len': 4096, 'tensor_parallel_size': 2, 'gpu_memory_utilization': 0.5, 'disable_log_stats': True, 'model': '../llms/Qwen2.5-7B-Instruct'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


INFO 12-28 21:10:37 [__init__.py:742] Resolved architecture: Qwen2ForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


WARNING 12-28 21:10:37 [__init__.py:2767] Casting torch.bfloat16 to torch.float16.
INFO 12-28 21:10:37 [__init__.py:1815] Using max model len 4096


...

(Worker_TP1 pid=458460) INFO 12-28 21:10:57 [default_loader.py:268] Loading weights took 6.08 seconds
(Worker_TP0 pid=458459) INFO 12-28 21:10:57 [default_loader.py:268] Loading weights took 6.27 seconds


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:06<00:00,  1.58s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:06<00:00,  1.55s/it]
(Worker_TP0 pid=458459) 


(Worker_TP1 pid=458460) INFO 12-28 21:10:57 [gpu_model_runner.py:2392] Model loading took 7.1217 GiB and 6.254890 seconds
(Worker_TP0 pid=458459) INFO 12-28 21:10:57 [gpu_model_runner.py:2392] Model loading took 7.1217 GiB and 6.461771 seconds
(Worker_TP1 pid=458460) INFO 12-28 21:11:01 [backends.py:539] Using cache directory: /home/idea_user/.cache/vllm/torch_compile_cache/ee00c9d802/rank_1_0/backbone for vLLM's torch.compile
(Worker_TP1 pid=458460) INFO 12-28 21:11:01 [backends.py:550] Dynamo bytecode transform time: 3.93 s
(Worker_TP0 pid=458459) INFO 12-28 21:11:02 [backends.py:539] Using cache directory: /home/idea_user/.cache/vllm/torch_compile_cache/ee00c9d802/rank_0_0/backbone for vLLM's torch.compile
(Worker_TP0 pid=458459) INFO 12-28 21:11:02 [backends.py:550] Dynamo bytecode transform time: 4.35 s
(Worker_TP1 pid=458460) INFO 12-28 21:11:03 [backends.py:194] Cache the graph for dynamic shape for later use
(Worker_TP0 pid=458459) INFO 12-28 21:11:04 [backends.py:194] Cache th

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 67/67 [00:02<00:00, 26.35it/s]


(Worker_TP0 pid=458459) INFO 12-28 21:11:27 [custom_all_reduce.py:203] Registering 3819 cuda graph addresses
(Worker_TP1 pid=458460) INFO 12-28 21:11:28 [custom_all_reduce.py:203] Registering 3819 cuda graph addresses
(Worker_TP0 pid=458459) INFO 12-28 21:11:28 [gpu_model_runner.py:3118] Graph capturing finished in 4 secs, took 0.48 GiB
(Worker_TP0 pid=458459) INFO 12-28 21:11:28 [gpu_worker.py:391] Free memory on device (65.96/79.32 GiB) on startup. Desired GPU memory utilization is (0.5, 39.66 GiB). Actual usage is 7.12 GiB for weight, 1.45 GiB for peak activation, 1.09 GiB for non-torch memory, and 0.48 GiB for CUDAGraph memory. Replace gpu_memory_utilization config with `--kv-cache-memory=31540551680` to fit into requested memory, or `--kv-cache-memory=59777359872` to fully utilize gpu memory. Current kv cache memory in use is 32209543168 bytes.
(Worker_TP1 pid=458460) INFO 12-28 21:11:29 [gpu_model_runner.py:3118] Graph capturing finished in 4 secs, took 0.48 GiB
(Worker_TP1 pid=4

In [11]:
list1 = [10, 20, 23, 24, 27, 29, 44, 57, 58, 59, 64, 70, 72, 73, 76, 77, 80, 81, 89, 90, 95, 100, 101, 103, 106, 112, 114, 119, 120, 123, 126, 127, 129, 130, 132, 135, 137, 141, 152, 154, 159, 166, 169, 171, 174, 175, 176, 177, 178, 179, 182, 186, 187, 191, 196, 197, 205, 206, 207, 208, 209, 210, 211, 226, 229, 230, 232, 234, 235, 236, 240, 247, 250, 252, 258, 259, 261, 262, 263, 270, 271, 272, 275, 276, 281, 284, 285, 289, 290, 297, 300, 302, 306, 307, 308, 315, 317, 318, 320, 322, 323, 325, 330, 332, 345, 351, 354, 359, 361, 363, 366, 368, 370, 371, 372, 374, 380, 386, 388, 389, 391, 394, 399, 405, 406, 407, 408, 410, 413, 414, 416, 419, 420, 421, 423, 424, 427, 428, 429, 430, 433, 444, 446, 448, 453, 454, 456, 459, 464, 468, 481, 482, 484, 486, 487, 488, 494, 503, 504, 509, 510, 512, 516, 521, 531, 537, 545, 546, 547, 552, 553, 558, 566, 583, 597]
list2 = [10, 12, 16, 19, 23, 27, 29, 30, 35, 39, 44, 46, 49, 51, 58, 61, 64, 70, 72, 76, 77, 80, 81, 83, 89, 90, 92, 94, 95, 96, 99, 100, 101, 103, 106, 109, 112, 114, 116, 119, 120, 123, 126, 127, 129, 130, 132, 135, 141, 148, 152, 153, 154, 158, 159, 166, 172, 174, 175, 176, 177, 178, 179, 182, 186, 187, 191, 197, 200, 201, 204, 205, 206, 207, 208, 209, 210, 226, 228, 229, 230, 232, 235, 236, 240, 247, 252, 255, 258, 259, 261, 263, 270, 271, 272, 275, 276, 280, 281, 284, 285, 289, 290, 297, 300, 302, 303, 308, 312, 315, 318, 325, 326, 328, 330, 332, 334, 341, 345, 347, 348, 351, 359, 361, 363, 366, 368, 370, 371, 372, 373, 374, 385, 386, 387, 389, 396, 399, 403, 405, 407, 408, 413, 414, 416, 419, 421, 424, 427, 428, 429, 433, 435, 437, 445, 448, 453, 456, 458, 459, 464, 476, 481, 482, 486, 487, 488, 490, 494, 501, 503, 504, 506, 509, 510, 511, 518, 521, 523, 524, 530, 531, 537, 542, 546, 547, 549, 550, 551, 553, 565, 566, 567, 576, 578, 583, 585, 589, 599]
list3 = [10, 16, 19, 23, 26, 27, 29, 44, 46, 49, 51, 57, 58, 64, 66, 70, 72, 76, 78, 80, 81, 89, 90, 95, 100, 103, 106, 109, 112, 114, 116, 119, 120, 122, 123, 127, 129, 135, 137, 141, 142, 152, 154, 168, 169, 172, 174, 175, 176, 177, 178, 182, 185, 187, 191, 196, 197, 205, 206, 207, 208, 209, 210, 226, 229, 230, 232, 235, 236, 240, 245, 250, 252, 258, 259, 260, 261, 270, 271, 272, 275, 276, 284, 289, 290, 293, 297, 302, 308, 315, 318, 323, 325, 328, 330, 334, 340, 345, 351, 352, 354, 359, 361, 363, 368, 370, 371, 372, 374, 385, 386, 388, 389, 391, 392, 394, 396, 400, 405, 406, 407, 408, 410, 413, 414, 419, 421, 423, 424, 427, 428, 429, 433, 435, 445, 448, 453, 456, 459, 464, 468, 481, 482, 484, 486, 487, 488, 492, 494, 503, 506, 509, 510, 511, 518, 521, 524, 525, 531, 537, 546, 547, 552, 553, 560, 565, 566, 567, 576, 583, 596, 597]
list4 = [5, 10, 13, 16, 19, 27, 29, 33, 44, 46, 49, 51, 58, 62, 64, 70, 72, 76, 77, 78, 89, 90, 94, 95, 100, 101, 103, 105, 112, 116, 119, 120, 123, 126, 127, 129, 135, 137, 152, 154, 166, 174, 175, 176, 177, 178, 179, 182, 186, 191, 196, 197, 204, 205, 206, 207, 208, 209, 210, 217, 226, 229, 230, 232, 234, 235, 236, 240, 252, 255, 258, 259, 260, 261, 262, 263, 265, 270, 271, 272, 275, 276, 278, 281, 284, 285, 289, 290, 297, 300, 302, 307, 308, 311, 323, 325, 326, 328, 332, 337, 340, 341, 345, 348, 351, 354, 359, 361, 363, 366, 368, 370, 372, 374, 386, 388, 389, 392, 396, 400, 405, 406, 407, 408, 410, 413, 414, 421, 422, 423, 424, 427, 429, 433, 436, 444, 445, 448, 453, 456, 459, 464, 481, 482, 486, 487, 488, 494, 501, 504, 506, 509, 510, 511, 518, 521, 523, 524, 525, 531, 537, 546, 547, 550, 552, 553, 558, 560, 566, 569, 578, 583, 585, 596, 597, 598]
inter = set(list1).intersection(set(list2).intersection(set(list3)).intersection(set(list4)))
print(len(inter), len(list1),len(list2))

102 175 199


In [ ]:
question = "When were the Normans in Normandy?"
question2 = "Who was the Norse leader?"
context = "The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse (\"Norman\" comes from \"Norseman\") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries."
system_prompt = ""
full_text1 = "Context:"+context+"\nQuestion:"+question+"\nAnswer:"
full_text2 = "Context:"+context+"\nQuestion:"+question2+"\nAnswer:"

messages1 = [{"role":"system", "content":system_prompt},
            {"role":"user", "content":full_text1}]
messages2 = [{"role":"system", "content":system_prompt},
            {"role":"user", "content":full_text2}]
text1 = tokenizer.apply_chat_template(messages1, add_generation_prompt=True, tokenize=False)
text2 = tokenizer.apply_chat_template(messages2, add_generation_prompt=True, tokenize=False)

print(text1)
print(text2)
device = hf_model.device
# print(device)
model_inputs1 = tokenizer([text1], return_tensors="pt").to(device)
model_inputs2 = tokenizer([text2], return_tensors="pt").to(device)
print(model_inputs1.input_ids.shape)
print(model_inputs2.input_ids.shape)
mask_length1 = tokenizer([text1.split("Question:")[0]],return_tensors="pt").input_ids.shape[1]
print(mask_length1)
test_length = model_inputs1.input_ids.shape[1]
print("test_length:", test_length)

# padding之后看下长度
text_list = [text1, text2]
model_input_list = tokenizer(text_list, return_tensors="pt",padding=True)
print(model_input_list.input_ids.shape)
print(len(text1), len(text2))
# print("-"*16)
# text = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=True)
# print(text)
# 利用模型进行生成
# generated_ids1 = model.generate(**model_inputs, max_new_tokens=512)
# generated_ids2 = model(**model_inputs)
# print(generated_ids1)
# print(generated_ids2)

<|im_start|>system
<|im_end|>
<|im_start|>user
Context:The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.
Question:When were the Normans in Normandy?
Answer:<|im_end|>
<|im_start|>assistant

<|im_start|>system
<|im_end|>
<|im_start|>user
Context:The Normans (Norman: Nourmands; French: Normands; Latin: Norma

In [5]:
test = "we are the champion. Context: this is the context, questionm....Context: this is the second context"
first = test.find("Context:")
print(first, type(first))
second = test.find("Context:", first+1)
print(second)
print(test[second:])

21 <class 'int'>
64
Context: this is the second context


In [6]:
model_inputs1 = tokenizer([text1], return_tensors="pt", padding=True).to(device)
print(model_inputs1.input_ids.shape)
model_inputs2 = tokenizer(text1, return_tensors="pt").to(device)
print(model_inputs2.input_ids.shape)
input_ids = model_inputs1.input_ids.tolist()
print(len(input_ids))

torch.Size([1, 195])
torch.Size([1, 195])
1


In [7]:
# hf_output = hf_model(**model_inputs)
# print(hf_output)

# 针对hf版本，计算loss
hf_inputs = tokenizer([text1], padding=True, return_tensors="pt", truncation=True).to(device)
# print(hf_inputs)
hf_outputs = hf_model(**hf_inputs)
print(hf_outputs.logits.shape)
shift_logits = hf_outputs.logits[...,:-1,:].contiguous()
shift_labels = hf_inputs["input_ids"][...,1:].contiguous()
print(shift_logits.shape)
print(shift_labels.shape)
loss_fct = torch.nn.CrossEntropyLoss(reduction="none", ignore_index=tokenizer.pad_token_id)

# 展开进行计算
print("----computing----")
shift_logits = shift_logits.view(-1, shift_logits.size(-1))
print(shift_logits.shape)
loss = loss_fct(shift_logits, shift_labels.view(-1)).view(shift_labels.size())
print("----loss results----")
print(loss.shape)   # 处理之后loss的长度和label是一样的
# print(loss)
ori_loss = torch.sum(loss,1)
print(ori_loss)
mask_length = len(tokenizer(text1, verbose=False)["input_ids"])
print(mask_length)
# 按照mask length进行处理
print("the mask length and the test length are: ", mask_length, test_length)
mask = torch.zeros_like(shift_labels)
for i in range(len(mask)):
    for j in range(mask_length, test_length-1):
        mask[i][j] = 1
    loss = loss* mask
ce_loss = torch.sum(loss, 1)
print(ce_loss)

torch.Size([1, 195, 152064])
torch.Size([1, 194, 152064])
torch.Size([1, 194])
----computing----
torch.Size([194, 152064])
----loss results----
torch.Size([1, 194])
tensor([311.1015], device='cuda:0', grad_fn=<SumBackward1>)
195
the mask length and the test length are:  195 195
tensor([0.], device='cuda:0', grad_fn=<SumBackward1>)


In [8]:
# 计算一下没有context的loss
messages = [{"role":"system", "content":system_prompt},
            {"role":"user", "content":question}]
text = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
model_inputs_q = tokenizer([text], return_tensors="pt").to(device)
print(model_inputs_q.input_ids.shape)
model_output = hf_model(**model_inputs_q)
print(model_output.logits.shape)
shift_logits = model_output.logits[...,:-1,:].contiguous()
shift_labels = model_inputs_q.input_ids[...,1:].contiguous()
print(shift_logits.shape, shift_labels.shape)
loss_fct = torch.nn.CrossEntropyLoss(reduction="none",ignore_index=tokenizer.pad_token_id)
shift_logits = shift_logits.view(-1, shift_logits.size(-1))
print("-------", shift_logits.shape)
loss = loss_fct(shift_logits, shift_labels.view(-1)).view(shift_labels.size())
print(loss.shape)
loss = torch.sum(loss, 1)
print(loss)
# 

torch.Size([1, 22])
torch.Size([1, 22, 152064])
torch.Size([1, 21, 152064]) torch.Size([1, 21])
------- torch.Size([21, 152064])
torch.Size([1, 21])
tensor([150.3349], device='cuda:0', grad_fn=<SumBackward1>)


In [9]:
# vllm版本的计算流程 
vllm_params = SamplingParams(temperature=0, max_tokens=1, prompt_logprobs=1, detokenize=False)
full_text = full_text1
question = question
# print(full_text)
# print("----")
# print(question)
full_messages = [{"role":"system", "content":system_prompt},
            {"role":"user", "content":full_text}]
text1 = tokenizer.apply_chat_template(full_messages, add_generation_prompt=True, tokenize=False)
# print(full_text)
# print(question)
question_messages = [{"role":"system", "content":system_prompt},
            {"role":"user", "content":question}]
question_text = tokenizer.apply_chat_template(question_messages, add_generation_prompt=True, tokenize=False)
# print(question_text)
vllm_inputs_full = tokenizer([text1], return_tensors="pt").to(device)
# print(full_text)
print(vllm_inputs_full.input_ids.shape)
mask_length = tokenizer([text1.split("Question")[0]], return_tensors="pt").input_ids.shape[1]
print(mask_length)
input_ids = vllm_inputs_full.input_ids.tolist()
print(type(input_ids),len(input_ids[0]))

# vllm generating
####！！！！需要注意的是！vllm输入是apply_chat_template之后的纯文本
vllm_outputs_full = vllm_model.generate(text1, sampling_params=vllm_params)
print(type(vllm_outputs_full[0].prompt_logprobs), len(vllm_outputs_full[0].prompt_logprobs))
full_log_probs=vllm_outputs_full[0].prompt_logprobs
print(type(full_log_probs), len(full_log_probs))

torch.Size([1, 195])
177
<class 'list'> 195


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  7.11it/s, est. speed input: 1407.70 toks/s, output: 7.22 toks/s]

<class 'list'> 195
<class 'list'> 195


In [10]:
print(input_ids)
print(full_log_probs[1])
# print(zip(input_ids[0], full_log_probs))

# 计算loss
def loss_cal(input_ids, outputs, mask_lengths):
    all_losses = []
    for ids, out in zip(input_ids, outputs):
        token_losses = []
        # print(ids)
        # print(out)
        # print("----")
        for tok_id, lp in zip(ids, out.prompt_logprobs):
            print(tok_id, lp)   # 第一个token的prompt_logprobs是none
            if lp is None:
                token_losses.append(0.0)
                continue
            info = lp.get(tok_id, None)
            if info is None:
                token_losses.append(20.0)    # fallback, 大loss
            else:
                token_losses.append(-info.logprob)
        # print(token_losses)
        all_losses.append(torch.tensor(token_losses))
    loss = torch.nn.utils.rnn.pad_sequence(all_losses, batch_first=True, padding_value=0.0)
    loss = loss[:, 1:]  # 与shift_losses对齐
    mask = torch.zeros_like(loss)
    for i in range(len(mask)):
        mask[i, mask_lengths[i]:] = 1
    loss = mask * loss
    ce_loss = torch.sum(loss, dim=1)
    return ce_loss

vllm_ce_loss = loss_cal(input_ids, vllm_outputs_full,[mask_length, ])

print(vllm_ce_loss)

# 针对没有context的计算
question_input_ids = tokenizer([question_text], return_tensors="pt").to(device)
print(question_input_ids.input_ids.shape)
question_outputs = vllm_model.generate(question_text, sampling_params=vllm_params)
print(len(question_outputs[0].prompt_logprobs))
question_ce_loss = loss_cal(question_input_ids, question_outputs, [0,])
print(question_ce_loss)

[[151644, 8948, 198, 151645, 198, 151644, 872, 198, 1972, 74045, 20317, 596, 320, 24993, 276, 25, 451, 413, 1928, 82, 26, 8585, 25, 20317, 2844, 26, 19458, 25, 20317, 42661, 8, 1033, 279, 1251, 879, 304, 279, 220, 16, 15, 339, 323, 220, 16, 16, 339, 23631, 6551, 862, 829, 311, 20317, 13331, 11, 264, 5537, 304, 9625, 13, 2379, 1033, 57742, 504, 93514, 3489, 24993, 276, 1, 4041, 504, 330, 45, 10836, 1515, 899, 24797, 388, 323, 61475, 504, 34340, 11, 38396, 323, 31503, 879, 11, 1212, 862, 7653, 431, 27961, 11, 7230, 311, 46077, 1152, 10026, 311, 6210, 14927, 14429, 315, 4312, 9694, 685, 13, 16904, 21787, 315, 38954, 13455, 323, 26792, 448, 279, 9867, 9270, 812, 323, 12751, 12010, 4943, 812, 21910, 11, 862, 48346, 1035, 26024, 10880, 448, 279, 10252, 287, 1103, 5980, 26735, 315, 4312, 9694, 685, 13, 576, 12460, 12752, 323, 21551, 9569, 315, 279, 20317, 596, 21992, 15102, 304, 279, 1156, 4279, 315, 279, 220, 16, 15, 339, 9294, 11, 323, 432, 8570, 311, 37580, 916, 279, 72720, 23631, 624, 145

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 57.96it/s, est. speed input: 1343.80 toks/s, output: 60.95 toks/s]

22
i None
n {8948: Logprob(logprob=-14.035704612731934, rank=28812, decoded_token=None), 15: Logprob(logprob=-2.0791616439819336, rank=1, decoded_token=None)}
p {198: Logprob(logprob=-2.4127628803253174, rank=1, decoded_token=None)}
u {151645: Logprob(logprob=-22.993486404418945, rank=151782, decoded_token=None), 198: Logprob(logprob=-2.290360927581787, rank=1, decoded_token=None)}
t {198: Logprob(logprob=-0.001869717612862587, rank=1, decoded_token=None)}
_ {151644: Logprob(logprob=-19.759784698486328, rank=100458, decoded_token=None), 750: Logprob(logprob=-1.984393835067749, rank=1, decoded_token=None)}
i {872: Logprob(logprob=-13.168075561523438, rank=97, decoded_token=None), 198: Logprob(logprob=-0.004013341851532459, rank=1, decoded_token=None)}
d {198: Logprob(logprob=-1.4294354915618896, rank=2, decoded_token=None), 271: Logprob(logprob=-1.1638104915618896, rank=1, decoded_token=None)}
s {4498: Logprob(logprob=-5.536229133605957, rank=37, decoded_token=None), 3838: Logprob(logpr

In [162]:
# 测试一下不同sampling参数对vllm生成结果的影响
sampling2= SamplingParams(temperature=0, max_tokens=1,top_p=1, top_k=1)
output2 = vllm_model.generate(question_text, sampling_params=sampling2)
print(output2)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 67.33it/s, est. speed input: 1574.98 toks/s, output: 71.43 toks/s]

[RequestOutput(request_id=30, prompt='<|im_start|>system\n<|im_end|>\n<|im_start|>user\nWhen were the Normans in Normandy?<|im_end|>\n<|im_start|>assistant\n', prompt_token_ids=[151644, 8948, 198, 151645, 198, 151644, 872, 198, 4498, 1033, 279, 20317, 596, 304, 20317, 13331, 30, 151645, 198, 151644, 77091, 198], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='The', token_ids=[785], cumulative_logprob=None, logprobs=None, finish_reason=length, stop_reason=None)], finished=True, metrics=None, lora_request=None, num_cached_tokens=16, multi_modal_placeholders={})]


In [ ]:
# 尝试用vllm生成ce计算过程
all_losses = []
input_ids = model_inputs["input_ids"].tolist()
print(input_ids[0])
print(len(input_ids))

# for ids, out in zip(model_inputs, xx):
    # pass

[151644, 8948, 198, 151645, 198, 151644, 872, 198, 574, 374, 264, 1273, 1467, 151645, 198, 151644, 77091, 198]
1


In [21]:
print(hf_output.logits.shape)
print(vllm_outputs[0].outputs[0].logprobs)

torch.Size([1, 18, 152064])
[{284: Logprob(logprob=-1.1678533554077148, rank=1, decoded_token=' ='), 11: Logprob(logprob=-2.230353355407715, rank=2, decoded_token=','), 788: Logprob(logprob=-2.441290855407715, rank=3, decoded_token='":'), 1210: Logprob(logprob=-2.785040855407715, rank=4, decoded_token="':"), 63: Logprob(logprob=-2.956915855407715, rank=5, decoded_token='`')}, {7834: Logprob(logprob=-1.137711524963379, rank=1, decoded_token=' torch'), 45958: Logprob(logprob=-2.059586524963379, rank=2, decoded_token=' tokenizer'), 1946: Logprob(logprob=-2.239274024963379, rank=3, decoded_token=' input'), 508: Logprob(logprob=-2.543961524963379, rank=4, decoded_token=' ['), 4167: Logprob(logprob=-3.137711524963379, rank=5, decoded_token=' []\n')}, {40588: Logprob(logprob=-0.8098697066307068, rank=1, decoded_token='.tensor'), 41811: Logprob(logprob=-1.4192447662353516, rank=2, decoded_token='.Long'), 23280: Logprob(logprob=-2.7317447662353516, rank=3, decoded_token='.randint'), 32632: Logp

In [18]:
print(type(vllm_outputs))
print(len(vllm_outputs))
print(vllm_outputs[0])
print(vllm_outputs[1])


<class 'list'>
2
RequestOutput(request_id=3, prompt='input_ids', prompt_token_ids=[1355, 8077], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=[None, {8077: Logprob(logprob=-7.443201065063477, rank=88, decoded_token=None), 284: Logprob(logprob=-1.1267945766448975, rank=1, decoded_token=None), 3904: Logprob(logprob=-2.4705445766448975, rank=2, decoded_token=None), 16: Logprob(logprob=-2.7908570766448975, rank=3, decoded_token=None), 2458: Logprob(logprob=-2.8221070766448975, rank=4, decoded_token=None), 1703: Logprob(logprob=-3.4783570766448975, rank=5, decoded_token=None), 741: Logprob(logprob=-3.5017945766448975, rank=6, decoded_token=None), 2895: Logprob(logprob=-3.9236695766448975, rank=7, decoded_token=None), 2019: Logprob(logprob=-4.040857315063477, rank=8, decoded_token=None), 5500: Logprob(logprob=-4.095544815063477, rank=9, decoded_token=None), 703: Logprob(logprob=-4.439294815063477, rank=10, decoded_token=None), 445: Logprob(logprob=-4.462732315063477, ra

In [29]:
print(type(generated_ids1), generated_ids1.shape)
print(type(generated_ids2), len(generated_ids2["logits"].shape))
print(generated_ids2.logits.shape)
print(model_inputs.input_ids.shape)
print(tokenizer.vocab_size)
print(tokenizer.pad_token_id)
print(tokenizer.pad_token_type_id)

<class 'torch.Tensor'> torch.Size([1, 52])
<class 'transformers.modeling_outputs.CausalLMOutputWithPast'> 3
torch.Size([1, 18, 152064])
torch.Size([1, 18])
151643
151643
0


In [ ]:
shift_logits = generated_ids2.logits[...,:-1,:].contiguous()
shift_labels = model_inputs["input_ids"][...,1:].contiguous()

loss_fct = torch.nn.CrossEntropyLoss(reduction="none", ignore_index=tokenizer.pad_token_id)
# reduction="none"，会返回每一个token的loss，而不是平均loss
shift_logits = shift_logits.view(-1, shift_logits.size(-1))
loss = loss_fct(shift_logits, shift_labels.view(-1)).view(shift_labels.size())
mask_length = model_inputs.input_ids
print(len(model_inputs["input_ids"]))


1


In [39]:
old_input = tokenizer(text)["input_ids"]
print(len(old_input))
new_input = tokenizer(text, verbose=False)["input_ids"]
print(len(new_input))

18
18
